In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv("sentimentdataset.csv")
df.shape

(732, 15)

In [23]:
df = df[["Text","Sentiment"]]
df["Sentiment"] = df["Sentiment"].str.strip()

In [ ]:
x=sorted(df['Sentiment'].unique())
len(x)

In [ ]:
x

In [26]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["SentimentEncoded"] = label_encoder.fit_transform(df["Sentiment"])

In [27]:
from sklearn.model_selection import train_test_split
X = df["Text"]
y = df["SentimentEncoded"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)